In [1]:
import cv2 as cv
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import glob

In [2]:
import pydicom as pdi
import tensorflow_io as tfio

In [3]:
dicom_path = r'D:\Gproject\DataSet\test_data\CBIS-DDSM\*\*\*\*'
label_path = r'D:\Gproject\DataSet\test_data'

In [42]:
dicom_mask_path = r'D:\Gproject\DataSet\test_data\CBIS-DDSM\*\*\*\1-2.dcm'


In [87]:
dicom_mask_path_00 = glob.glob(dicom_mask_path)[0]
dicom_mask_path_00_ID = dicom_mask_path_00.split('\\')[5]
mask_ID, mask_slice = dicom_mask_path_00_ID[2], dicom_mask_path_00_ID[4]

In [61]:
dicom_full_path_00 = glob.glob(dicom_full_path)[0]
dicom_full_path_00_ID = dicom_full_path_00.split('\\')[5].split("_")
full_ID, full_slice = dicom_full_path_00_ID[2], dicom_full_path_00_ID[4]



In [108]:

# dicom path를 입력으로 받아서 img반환 (ndarray)
# normalize추가(full 이미지 normalize)
def dcm2img_norm(dcm_path):
    image_bytes = tf.io.read_file(dcm_path)
    
    decode_img_lossy = tfio.image.decode_dicom_image(image_bytes,scale='auto',on_error='lossy', dtype=tf.uint8)
    #############################################################################################
    # args
    # scale : datatype 변형시 fill stretching 
    # on_error : dtype uint 16 -> 8할때 error 방지
    #############################################################################################
    decode_img_lossy_ndr = decode_img_lossy.numpy()
    decode_img_lossy_ndr_norm = decode_img_lossy_ndr[0,...,0]/255
    #decode_img_lossy_ndr_norm_add_axis = decode_img_lossy_ndr_norm[np.newaxis,...,np.newaxis]
    return decode_img_lossy_ndr_norm

# dicom path를 입력으로 받아서 img반환 (ndarray)
# normalzize 미포함(mask 이미지)
def dcm2img(dcm_path):
    image_bytes = tf.io.read_file(dcm_path)
    
    decode_img_lossy = tfio.image.decode_dicom_image(image_bytes,scale='auto',on_error='lossy', dtype=tf.uint8)
    #############################################################################################
    # args
    # scale : datatype 변형시 fill stretching 
    # on_error : dtype uint 16 -> 8할때 error 방지
    #############################################################################################
    decode_img_lossy_ndr = decode_img_lossy.numpy()
    #decode_img_lossy_ndr_norm = decode_img_lossy_ndr[0,...,0]/255
    #decode_img_lossy_ndr_norm_add_axis = decode_img_lossy_ndr_norm[np.newaxis,...,np.newaxis]
    return decode_img_lossy_ndr

In [92]:
dsize = (1500,2300)

# input : dcm full mammography
# output : png file

dicom_full_path = r'D:\Gproject\DataSet\tcia_data\CBIS-DDSM\*\*\*full*\1-1.dcm'
output_path = r'D:\Gproject\DataSet\tcia_data\imgs'
count = 0
for dcm in glob.glob(dicom_full_path):
    
    # 파일명 slicing
    dicom_full_path_ID = dcm.split('\\')[5]
    
    # dcm -> img(ndrrray)
    img = dcm2img_norm(dcm)
    
    # TODO : resize 가능한가
    img_resize = cv.resize(img,dsize,interpolation = cv.INTER_AREA)
    
    # 저장파일명
    output_name = output_path + "\\"+ dicom_full_path_ID + "_full"+ ".png"
    plt.imsave(output_name, img_resize,cmap = 'gray')

    
# input : dcm mask mammography
# output : png file

dicom_mask_path = r'D:\Gproject\DataSet\tcia_data\CBIS-DDSM\*\*\*mask*\1-2.dcm'
output_path = r'D:\Gproject\DataSet\tcia_data\imgs'
count = 0
for dcm in glob.glob(dicom_mask_path):
    
    # 파일명 slicing
    dicom_mask_path_ID = dcm.split('\\')[5]
    
    # dcm -> img(ndrrray)
    img = dcm2img(dcm)
    
    # dicom mass train 에서 1-1.dcm과 1-2.dcm사이 매칭이 잘못되있음 --> 해결
    ##################################################
    if img.shape[0] < 2000 or img.shape[1] < 2000:
        print("*Changed*")
        str_list = dcm.split('\\')
        str_list.pop()
        str_list.append('1-1.dcm')
        dcm = str_list[0]+"\\" + str_list[1]+"\\" + str_list[2]+"\\" + str_list[3]+"\\"  + str_list[4] +"\\" + str_list[5]+"\\" + str_list[6]+"\\" + str_list[7]+"\\" + str_list[8] 
        img = dcm2img_norm(dcm)
    ##################################################
    
    #full 이미지와 동일한 크기로 resize
    # TODO : resize 가능한가
    img_resize = cv.resize(img,dsize,interpolation = cv.INTER_AREA)
    
    # 저장파일명
    output_name = output_path + "\\"+ dicom_mask_path_ID + "_mask"+ ".png"
    plt.imsave(output_name, img_resize,cmap = 'gray')
    
    print("Src path :", dcm)
    print("Dst path :", output_name)
    print("img shape : ", img.shape)
    print("img resize : ", img_resize.shape)
    print("\n\n")
    